In [35]:
import ee
import numpy as np
import pandas as pd

ee.Authenticate()
ee.Initialize(project="shrubwise-dc-488219")

RAP_veg_yearly_10m = ee.ImageCollection("projects/rap-data-365417/assets/vegetation-cover-10m")

In [36]:
aoi_bliss = ee.Geometry.Rectangle(
    [-120.1018846, 38.99274873, -120.0899834, 39.0020357],
    geodesic=False
)

minx, miny, maxx, maxy = -120.1018846, 38.99274873, -120.0899834, 39.0020357
center_lat = (miny + maxy) / 2
km_shift = 10
dlon = km_shift / (111.32 * np.cos(np.deg2rad(center_lat)))
aoi_demo_shifted = ee.Geometry.Rectangle([minx + dlon, miny, maxx + dlon, maxy], geodesic=False)

AOIS_DEFAULT = {
    "DL_Bliss": aoi_bliss,
    "Demo_Shifted_10km": aoi_demo_shifted,
}

In [37]:
def rap_10m_mosaic_for_year(year):
    return RAP_veg_yearly_10m.filter(ee.Filter.eq("year", year)).mosaic()

def rap_shrub_cover_10m(year):
    return rap_10m_mosaic_for_year(year).select("SHR").toFloat().rename("shrub_cover").set("year", year)

In [38]:
def rap_prior_feature_cube(
    year = 2025,
    years_for_mean = range(2018, 2026),
    ge_thresholds = (5, 10),
    add_distance_priors = True
):
    shrub_y = rap_shrub_cover_10m(year).rename(f"rap_shrub_{year}")

    stack = ee.ImageCollection([rap_shrub_cover_10m(y) for y in years_for_mean])
    mean_img = stack.mean().rename(f"rap_shrub_mean_{min(years_for_mean)}_{max(years_for_mean)}")
    std_img = stack.reduce(ee.Reducer.stdDev()).rename(f"rap_shrub_std_{min(years_for_mean)}_{max(years_for_mean)}")

    def add_t_float(im):
        t = ee.Image.constant(ee.Number(im.get("year"))).toFloat().rename("t")
        yb = im.select("shrub_cover").toFloat()
        return yb.addBands(t)

    fit = stack.map(add_t_float).select(["t", "shrub_cover"]).reduce(ee.Reducer.linearFit())
    trend_img = fit.select("scale").rename(f"rap_shrub_trend_per_year_{min(years_for_mean)}_{max(years_for_mean)}")

    masks = []
    for th in ge_thresholds:
        masks.append(shrub_y.gte(th).rename(f"rap_shrub_ge{th}_{year}"))

    out = ee.Image.cat([shrub_y, mean_img, std_img, trend_img] + masks)

    # Distance-to-shrub priors
    if add_distance_priors:
        max_radius_px = 255
    
        for th in ge_thresholds:
            zone = shrub_y.gte(th).selfMask()
    
            dist_px = zone.Not().distance(ee.Kernel.euclidean(max_radius_px, "pixels"))
            dist_m = dist_px.multiply(10).rename(f"rap_dist_to_ge{th}_{year}_m")
    
            out = out.addBands(dist_m)

    return out

In [39]:
prior_cube = rap_prior_feature_cube()
print(prior_cube.bandNames().getInfo())

['rap_shrub_2025', 'rap_shrub_mean_2018_2025', 'rap_shrub_std_2018_2025', 'rap_shrub_trend_per_year_2018_2025', 'rap_shrub_ge5_2025', 'rap_shrub_ge10_2025', 'rap_dist_to_ge5_2025_m', 'rap_dist_to_ge10_2025_m']


In [40]:
def summarize_priors_over_aois(image, aois, scale=10):
    bands = image.bandNames().getInfo()
    rows = []

    reducer = (
        ee.Reducer.count()
          .combine(ee.Reducer.mean(), sharedInputs=True)
          .combine(ee.Reducer.median(), sharedInputs=True)
          .combine(ee.Reducer.min(), sharedInputs=True)
          .combine(ee.Reducer.max(), sharedInputs=True)
    )

    for aoi_name, geom in aois.items():
        stats = image.reduceRegion(reducer, geom, scale, maxPixels=1e9, bestEffort=True).getInfo()

        row = {"aoi": aoi_name}
        for b in bands:
            row[f"{b}_count"] = stats.get(f"{b}_count")
            row[f"{b}_mean"]  = stats.get(f"{b}_mean")
            row[f"{b}_median"]= stats.get(f"{b}_median")
            row[f"{b}_min"]   = stats.get(f"{b}_min")
            row[f"{b}_max"]   = stats.get(f"{b}_max")

        rows.append(row)

    return pd.DataFrame(rows).set_index("aoi")

In [41]:
summary = summarize_priors_over_aois(prior_cube, AOIS_DEFAULT, scale=10)
summary

,rap_shrub_2025_count,rap_shrub_2025_mean,rap_shrub_2025_median,rap_shrub_2025_min,rap_shrub_2025_max,rap_shrub_mean_2018_2025_count,rap_shrub_mean_2018_2025_mean,rap_shrub_mean_2018_2025_median,rap_shrub_mean_2018_2025_min,rap_shrub_mean_2018_2025_max,...,rap_dist_to_ge5_2025_m_count,rap_dist_to_ge5_2025_m_mean,rap_dist_to_ge5_2025_m_median,rap_dist_to_ge5_2025_m_min,rap_dist_to_ge5_2025_m_max,rap_dist_to_ge10_2025_m_count,rap_dist_to_ge10_2025_m_mean,rap_dist_to_ge10_2025_m_median,rap_dist_to_ge10_2025_m_min,rap_dist_to_ge10_2025_m_max
aoi,,,,,,,,,,,,,,,,,,,,,
DL_Bliss,13936,2.428642,0,0,24,13936,2.865405,0,0,25.125,...,0,None,None,None,None,0,None,None,None,None
Demo_Shifted_10km,13936,0.000000,0,0,0,13936,0.000870,0,0,0.125,...,0,None,None,None,None,0,None,None,None,None


In [42]:
def make_shrub_strata(shrub_cover_img):
    s = shrub_cover_img.rename("shrub_cover").toFloat()
    strata = s.gt(0).add(s.gte(5)).rename("strata").toInt()
    return strata

In [46]:
def stratified_points_for_aoi(
    strata_img,
    region,
    points_per_class = 500,
    scale = 10,
    add_features_img = None,
):
    class_points = ee.Dictionary({
        0: points_per_class,
        1: points_per_class,
        2: points_per_class,
    })

    pts = strata_img.stratifiedSample(
        numPoints=points_per_class,
        classBand="strata",
        region=region,
        scale=scale,
        classPoints=class_points,
        geometries=True    
    )

    if add_features_img is not None:
        pts = add_features_img.sampleRegions(collection=pts, scale=scale, geometries=True)

    return pts

In [47]:
def stratified_points_across_aois(
    aois,
    year = 2025,
    points_per_class = 500,
    scale = 10,
    attach_prior_features = True
):
    shrub_y = rap_shrub_cover_10m(year)
    strata = make_shrub_strata(shrub_y)

    features_img = rap_prior_feature_cube(year=year) if attach_prior_features else None

    out = {}
    for name, geom in aois.items():
        pts = stratified_points_for_aoi(
            strata_img=strata,
            region=geom,
            points_per_class=points_per_class,
            scale=scale,
            add_features_img=features_img
        )
        out[name] = pts

    return out

In [48]:
pts_by_aoi = stratified_points_across_aois(
    aois=AOIS_DEFAULT,
    year=2025
)

In [49]:
def resample_to_reference(
    image: ee.Image,
    reference: ee.Image,
    *,
    continuous_bands: list[str],
    categorical_bands: list[str] = None
) -> ee.Image:
    """
    Resample an image to the projection of a reference image.
    """
    categorical_bands = categorical_bands or []
    ref_proj = reference.projection()

    cont = image.select(continuous_bands).resample("bilinear").reproject(ref_proj)
    if categorical_bands:
        cat = image.select(categorical_bands).resample("nearest").reproject(ref_proj)
        return ee.Image.cat([cont, cat])
    return cont

In [ ]:
# example (later):
# prior_1m = resample_to_reference(prior_cube, reference=naip_1m,
#                                 continuous_bands=[...],
#                                 categorical_bands=[...])